### Example code for extracting trees from a trained model file.

In [1]:
import torch
from snli.model import SNLIModel
from utils.vocab import Vocab
import torch.nn as nn
import torch.nn.functional as F

In [2]:
torch.set_grad_enabled(False)
vocab = Vocab.from_file('pretrained/vocab.pkl', add_pad=True, add_unk=True)
model = SNLIModel(prim_num_classes=3, num_words=len(vocab), word_dim=300, hidden_dim=300,
                  clf_hidden_dim=1024, clf_num_layers=1, bidirectional=True, dropout_prob=0.13,
                  use_leaf_rnn=True, intra_attention=True, use_batchnorm=True)
model.load_state_dict(torch.load('pretrained/model.pkl', map_location='cpu'))
model.eval()

SNLIModel(
  (word_embedding): Embedding(43439, 300)
  (encoder): BinaryTreeLSTM(
    (att_wt): Linear(in_features=600, out_features=1, bias=True)
    (pre_att): Linear(in_features=600, out_features=600, bias=True)
    (leaf_rnn_cell): LSTMCell(300, 300)
    (leaf_rnn_cell_bw): LSTMCell(300, 300)
    (treelstm_layer): BinaryTreeLSTMLayer(
      (comp_linear): Linear(in_features=1200, out_features=3000, bias=True)
    )
  )
  (classifier): SNLIClassifier(
    (bn_mlp_input): BatchNorm1d(2400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn_mlp_output): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (dropout): Dropout(p=0.13)
    (mlp): Sequential(
      (0): Sequential(
        (0): Linear(in_features=2400, out_features=1024, bias=True)
        (1): ReLU()
      )
    )
    (clf_linear): Linear(in_features=1024, out_features=3, bias=True)
  )
  (dropout): Dropout(p=0.13)
)

In [3]:
def prepare_input(sent_words):
    word_ids = torch.LongTensor([[vocab.word_to_id(w) for w in sent_words]])
    length = torch.LongTensor([len(sent_words)])
    word_ids_emb = model.word_embedding(word_ids)
    return word_ids_emb, length

In [6]:
from nltk.tokenize import word_tokenize
def get_embedding(sentence):
    words = word_tokenize(sentence.lower())
    enc_input, enc_length = prepare_input(words)
    h = model.encoder.forward(enc_input, enc_length, return_select_masks=True)[0]
    return h

In [7]:
h = get_embedding('I like to drink Orange Juice.') # Correct Answer
h1 = get_embedding('I love orange juice. ') # Acceptable answer
h2 = get_embedding('I like drinking orange juice. ')# Acceptable answer
h3 = get_embedding('I like to eat Oranges. ')# Non Acceptable answer
h4 = get_embedding('I do not like orange juice. ')#Non Acceptable answer

In [8]:
cos = nn.CosineSimilarity(dim=1, eps=1e-6)

In [9]:
print(h.shape)

torch.Size([1, 600])


In [10]:
print(cos(h,h1))
print(cos(h,h2))
print(cos(h,h3))
print(cos(h,h4))

tensor([0.8091])
tensor([0.9130])
tensor([0.6160])
tensor([0.5849])


In [ ]:
import spacy

nlp = spacy.load("en_core_web_md")  # make sure to use larger model!
tokens = nlp("dog cat banana")`

In [ ]:
!python3 -m spacy download en_core_web_md

In [ ]:
!pip3 install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz

In [ ]:
import spacy
import en_core_web_md

nlp = en_core_web_md.load()  # make sure to use larger model!
tokens = nlp("dog cat banana")

In [ ]:
h = nlp('I like to drink Orange Juice.')
h1 = nlp('I love orange juice. ')
h2 = nlp('I like drinking orange juice. ')
h3 = nlp('I like to eat Oranges. ')
h4 = nlp('I do not like orange juice. ')

In [ ]:
print(h.similarity(h1))
print(h.similarity(h2))
print(h.similarity(h3))
print(h.similarity(h4))